In [1]:
import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random

In [2]:
# Fix random seed
seed = 42
np.random.seed(seed)
random.seed(seed)
sampler = TPESampler(seed=seed)

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Pre-processing input data

In [4]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [5]:
results_df = pd.read_csv('../results_new/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio_new/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [6]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,45.91,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,25.77,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [7]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [8]:
mm_df = dataset_df[dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [9]:
st_df = dataset_df[~dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [10]:
# load test dataset
# g_test = pd.read_csv('csv/g_test.csv')
# st_test = pd.read_csv('csv/st_test.csv')
# mm_test = pd.read_csv('csv/mm_test.csv')

In [11]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target).drop(['benchmark','brand_raw', 'brand_raw_target'])
features_st = features.copy().drop(['count', 'count_target'])

In [12]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
x_g_scaler = MinMaxScaler(feature_range=(0, 1))
X_g_train = x_g_scaler.fit_transform(X_g_train)
X_g_test = x_g_scaler.transform(X_g_test)
y_g_scaler = MinMaxScaler(feature_range=(0, 1))
y_g_train = y_g_scaler.fit_transform(y_g_train.values.reshape(-1, 1)).flatten()
y_g_test = y_g_scaler.transform(y_g_test.values.reshape(-1, 1)).flatten()

In [13]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
x_st_scaler = MinMaxScaler(feature_range=(0, 1))
X_st_train = x_st_scaler.fit_transform(X_st_train)
X_st_test = x_st_scaler.transform(X_st_test)
y_st_scaler = MinMaxScaler(feature_range=(0, 1))
y_st_train = y_st_scaler.fit_transform(y_st_train.values.reshape(-1, 1)).flatten()
y_st_test = y_st_scaler.transform(y_st_test.values.reshape(-1, 1)).flatten()

In [14]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
x_mm_scaler = MinMaxScaler(feature_range=(0, 1))
X_mm_train = x_mm_scaler.fit_transform(X_mm_train)
X_mm_test = x_mm_scaler.transform(X_mm_test)
y_mm_scaler = MinMaxScaler(feature_range=(0, 1))
y_mm_train = y_mm_scaler.fit_transform(y_mm_train.values.reshape(-1, 1)).flatten()
y_mm_test = y_mm_scaler.transform(y_mm_test.values.reshape(-1, 1)).flatten()

# Model

In [15]:
def inv_scaling(y, y_scaler):
    return y_scaler.inverse_transform(y.reshape(-1, 1))

In [16]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test, y_scaler):
	dtrain = xgb.DMatrix(X_train, label=y_train)
	dtest = xgb.DMatrix(X_test, label=y_test)
	
	# Definimos los hiperparámetros a buscar (XGBoost)
	param = {
		"verbosity": 0,
		"device": "gpu",
		"objective": "reg:squarederror",
		"eval_metric": "rmse",
		"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
		# L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
		"learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
	}
    
	if param["booster"] in ["gbtree", "dart"]:
		# maximum depth of the tree, signifies complexity of the tree.
		param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
		# minimum child weight, larger the term more conservative the tree.
		param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
		param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
		# defines how selective algorithm is.
		param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
		param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

	if param["booster"] == "dart":
		param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
		param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
		param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
		param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

	# training
	model = xgb.train(param, dtrain)
	# evaluation
	preds = model.predict(dtest)
	mse_scaled = mean_squared_error(y_test, preds)
	mse = mean_squared_error(inv_scaling(y_test, y_scaler), inv_scaling(preds, y_scaler))
	rmse = np.sqrt(mse)
	mae = mean_absolute_error(inv_scaling(y_test, y_scaler), inv_scaling(preds, y_scaler))
	print(f"Trial: {trial.number} - MSE: {mse} - RMSE: {rmse} - MAE: {mae}")
	return mse_scaled

# Hyperparameters Optimization

In [17]:
n_trials = 100
study_g = None
study_st = None
study_mm = None

## General

In [18]:
# configuration optuna
study_g = optuna.create_study(direction='minimize', sampler=sampler)
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test, y_g_scaler), n_trials=n_trials)

Trial: 0 - MSE: 987.790185682391 - RMSE: 31.42912957245859 - MAE: 16.557445205853433
Trial: 1 - MSE: 1001.4878772945153 - RMSE: 31.646293263106113 - MAE: 16.676376172452358
Trial: 2 - MSE: 989.1930298208531 - RMSE: 31.451439232900825 - MAE: 16.56421613827267
Trial: 3 - MSE: 993.30851145354 - RMSE: 31.516797290548734 - MAE: 16.602446835285907
Trial: 4 - MSE: 996.4279226058027 - RMSE: 31.566246571390185 - MAE: 16.647650665901804


Trial: 5 - MSE: 993.9034794240918 - RMSE: 31.5262347803237 - MAE: 16.648870899922137
Trial: 6 - MSE: 1001.4878772945153 - RMSE: 31.646293263106113 - MAE: 16.676376172452358
Trial: 7 - MSE: 987.1253142210559 - RMSE: 31.418550479311676 - MAE: 16.61020111475764
Trial: 8 - MSE: 988.9816754113762 - RMSE: 31.44807904167401 - MAE: 16.620645110465382
Trial: 9 - MSE: 679.9651806647039 - RMSE: 26.076141981986215 - MAE: 15.53475800921466
Trial: 10 - MSE: 526.3832188955429 - RMSE: 22.94304293016824 - MAE: 15.036351154739792
Trial: 11 - MSE: 507.6879352717882 - RMSE: 22.53193145897147 - MAE: 14.938373671145051
Trial: 12 - MSE: 480.01388436396445 - RMSE: 21.90921916372111 - MAE: 14.880598751068115
Trial: 13 - MSE: 808.2436001477566 - RMSE: 28.429625395839402 - MAE: 15.935101360217944
Trial: 14 - MSE: 478.37684235554383 - RMSE: 21.871827595231814 - MAE: 14.189773867364831
Trial: 15 - MSE: 928.0582516802593 - RMSE: 30.464048510995042 - MAE: 16.26497575481518
Trial: 16 - MSE: 711.9120230282579 - RMSE: 

In [19]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 96
Mejores parametros: {'booster': 'dart', 'lambda': 8.454949858261484e-08, 'alpha': 2.958443409627235e-07, 'learning_rate': 0.0909507221631542, 'subsample': 0.8959347658610356, 'colsample_bytree': 0.8483727513060559, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.6139540352836711, 'gamma': 0.9107525035496924, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.9309378936570674e-05, 'skip_drop': 1.3380205255498052e-05}
Mejor valor de pérdida: 0.024950343190563686


## Single Thread

In [20]:
# configuration optuna
study_st = optuna.create_study(direction='minimize', sampler=sampler)
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test, y_st_scaler), n_trials=n_trials)

Trial: 0 - MSE: 1064.1945808639987 - RMSE: 32.621995353809965 - MAE: 16.23366529501638
Trial: 1 - MSE: 1103.3017138375524 - RMSE: 33.215985817638355 - MAE: 16.529044411936113
Trial: 2 - MSE: 1082.336915698488 - RMSE: 32.898889277580295 - MAE: 16.33358034416937
Trial: 3 - MSE: 991.8793342478538 - RMSE: 31.494115867060845 - MAE: 15.691470216812625
Trial: 4 - MSE: 1071.0287872265 - RMSE: 32.72657616107283 - MAE: 16.26521556829637
Trial: 5 - MSE: 580.4621564128688 - RMSE: 24.092782247238876 - MAE: 13.038607426821802
Trial: 6 - MSE: 942.008390000099 - RMSE: 30.6921551866287 - MAE: 15.301362894365862
Trial: 7 - MSE: 785.1822172802204 - RMSE: 28.021103070368596 - MAE: 13.861321516221569
Trial: 8 - MSE: 999.8900289005114 - RMSE: 31.62103775812096 - MAE: 15.717443522545599
Trial: 9 - MSE: 478.59366311619897 - RMSE: 21.87678365565192 - MAE: 12.037704514042021
Trial: 10 - MSE: 1088.3549632029446 - RMSE: 32.990225267538634 - MAE: 16.420683817463537
Trial: 11 - MSE: 372.3120087723378 - RMSE: 19.295

In [21]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 11
Mejores parametros: {'booster': 'gbtree', 'lambda': 0.0021535957566553797, 'alpha': 0.08596518740279382, 'learning_rate': 0.09708281774708119, 'subsample': 0.8307814364526688, 'colsample_bytree': 0.8887109093150083, 'max_depth': 7, 'min_child_weight': 9, 'eta': 2.4113747198291302e-08, 'gamma': 0.8835542456640489, 'grow_policy': 'depthwise'}
Mejor valor de pérdida en validación: 0.020981349834958165


## Multi Thread

In [22]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize', sampler=sampler)
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test, y_mm_scaler), n_trials=n_trials)

Trial: 0 - MSE: 700.3482069971574 - RMSE: 26.464092786210475 - MAE: 26.447168843587242
Trial: 1 - MSE: 565.767503184766 - RMSE: 23.78586771981981 - MAE: 23.770393976211544
Trial: 2 - MSE: 572.2918166349716 - RMSE: 23.922621441534613 - MAE: 23.906382911046343
Trial: 3 - MSE: 575.985425910368 - RMSE: 23.999696371212032 - MAE: 23.98324116134643
Trial: 4 - MSE: 556.2080890401476 - RMSE: 23.584064302832697 - MAE: 23.568480015436805
Trial: 5 - MSE: 562.3980763499537 - RMSE: 23.71493361470687 - MAE: 23.69944413566589
Trial: 6 - MSE: 579.9764359019724 - RMSE: 24.082699929658478 - MAE: 24.067213297526038
Trial: 7 - MSE: 552.0827618206087 - RMSE: 23.496441471435812 - MAE: 23.48080786132812
Trial: 8 - MSE: 555.9579222382611 - RMSE: 23.578759980929046 - MAE: 23.563163010915115
Trial: 9 - MSE: 583.2944161730646 - RMSE: 24.151488901785427 - MAE: 24.136279583613074
Trial: 10 - MSE: 552.0827618206087 - RMSE: 23.496441471435812 - MAE: 23.48080786132812
Trial: 11 - MSE: 552.0827618206087 - RMSE: 23.4964

In [23]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 81
Mejores parametros: {'booster': 'gblinear', 'lambda': 0.10550649323074619, 'alpha': 3.4087201612681223e-07, 'learning_rate': 0.09056969450973192, 'subsample': 0.4282198239630527, 'colsample_bytree': 0.2660203263926933}
Mejor valor de pérdida en validación: 0.13666949367750314


# Training

In [24]:
models_folder = '../models/xgboost'

In [25]:
#dump(scaler_g, f'{models_folder}/scaler_g.joblib')
#dump(scaler_st, f'{models_folder}/scaler_st.joblib')
#dump(scaler_mm, f'{models_folder}/scaler_mm.joblib')

## General

In [26]:
# hyperparameters
if study_g is not None:
	params = study_g.best_trial.params
else:
	params = {
		'booster': 'gbtree',
		'lambda': 6.153362802392881e-07,
		'alpha': 0.0011471508220748314,
		'learning_rate': 0.001002737827637667,
		'subsample': 0.20466176064508526,
		'colsample_bytree': 0.11320379521367623,
		'max_depth': 9,
		'min_child_weight': 2,
		'eta': 0.06983382228087344,
		'gamma': 7.814049704902547e-05,
		'grow_policy': 'lossguide'
	}

In [27]:
# general models
dtrain = xgb.DMatrix(X_g_train, label=y_g_train)
dtest = xgb.DMatrix(X_g_test, label=y_g_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(inv_scaling(y_g_test, y_g_scaler), inv_scaling(preds, y_g_scaler))
mae = mean_absolute_error(inv_scaling(y_g_test, y_g_scaler), inv_scaling(preds, y_g_scaler))
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 458.01439054419336 - RMSE: 21.40127076937707 - MAE: 13.495003354294234


In [28]:
# save model
model.save_model(f'{models_folder}/general.json')

## Single Thread

In [29]:
# hyperparameters
if study_st is not None:
	params = study_st.best_trial.params
else:
	params = {
		'booster': 'gbtree',
		'lambda': 1.932116425524344e-08,
		'alpha': 0.002607815095551034,
		'learning_rate': 0.0010967660905229464,
		'subsample': 0.27592137172367703,
		'colsample_bytree': 0.07486753519541206,
		'max_depth': 5,
		'min_child_weight': 9,
		'eta': 0.0027443109965453517,
		'gamma': 3.330037397996609e-08,
		'grow_policy': 'depthwise'
	}

In [30]:
# single thread model
dtrain = xgb.DMatrix(X_st_train, label=y_st_train)
dtest = xgb.DMatrix(X_st_test, label=y_st_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(inv_scaling(y_st_test, y_st_scaler), inv_scaling(preds, y_st_scaler))
mae = mean_absolute_error(inv_scaling(y_st_test, y_st_scaler), inv_scaling(preds, y_st_scaler))
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 368.44408669144303 - RMSE: 19.19489741289187 - MAE: 10.328600535928048


In [31]:
# save model
model.save_model(f'{models_folder}/single_thread.json')

## Multi Thread

In [32]:
# hyperparameters
if study_mm is not None:
	params = study_mm.best_trial.params
else:
	params = {
		'booster': 'gblinear',
		'lambda': 0.026758763366929273,
		'alpha': 0.985176822519382,
		'learning_rate': 0.05340673377810871,
		'subsample': 0.21906996068917262,
		'colsample_bytree': 0.15676169098118364
	}

In [33]:
# multi thread model
dtrain = xgb.DMatrix(X_mm_train, label=y_mm_train)
dtest = xgb.DMatrix(X_mm_test, label=y_mm_test)
# training
model = xgb.train(params, dtrain)
# evaluation
preds = model.predict(dtest)
mse = mean_squared_error(inv_scaling(y_mm_test, y_mm_scaler), inv_scaling(preds, y_mm_scaler))
mae = mean_absolute_error(inv_scaling(y_mm_test, y_mm_scaler), inv_scaling(preds, y_mm_scaler))
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 506.89576831297796 - RMSE: 22.51434583355639 - MAE: 22.49009060923258


In [34]:
# save model
model.save_model(f'{models_folder}/multi_thread.json')

# Load models

In [35]:
model_g = xgb.Booster()
model_st = xgb.Booster()
model_mm = xgb.Booster()
model_g.load_model(f'{models_folder}/general.json')
model_st.load_model(f'{models_folder}/single_thread.json')
model_mm.load_model(f'{models_folder}/multi_thread.json')

In [36]:
def describe_val(model, X, y, y_scaler):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}
	
	dtest = xgb.DMatrix(X)
	predictions = inv_scaling(model.predict(dtest), y_scaler)
	y_scaled = inv_scaling(y, y_scaler)
	index_min = np.argmin(np.abs(predictions - y_scaled))
	min_instance["prediction"] = predictions[index_min].item()
	min_instance["actual"] = y_scaled[index_min].item()
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y_scaled))
	max_instance["prediction"] = predictions[index_max].item()
	max_instance["actual"] = y_scaled[index_max].item()
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [37]:
# general model
print("Validation set general model")
min_instance, max_instance, predictions = describe_val(model_g, X_g_test, y_g_test, y_g_scaler)
y_scaled = inv_scaling(y_g_test, y_g_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("---")
print("Min instance")
print(g_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(g_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set general model
Mean prediction: 22.643465042114258 | Std actual: 13.231526374816895
Mean actual: 35.041486486486484 | Std actual: 28.83706408578472
Mean Error: 13.495003354294234 | Std Error: 16.61021598329717
---
Min instance
total_time                                                           13.47
total_cpu_usage                                                       0.99
max_ram_usage                                                  1436.714844
brand_raw                         Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz
count                                                                   12
l2_cache_size                                                          1.5
l3_cache_size                                                         12.0
l2_cache_line_size                                                     256
l2_cache_associativity                                                   6
benchmark                                                              KNP
ghz_actua

In [38]:
# single thread model
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val(model_st, X_st_test, y_st_test, y_st_scaler)
y_scaled = inv_scaling(y_st_test, y_st_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(st_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(st_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model
Mean prediction: 26.07541847229004 | Std actual: 14.80790901184082
Mean actual: 34.75261290322581 | Std actual: 31.49396398818138
Mean Error: 10.328600535928048 | Std Error: 16.179125367604698
Min instance
total_time                                                           13.47
total_cpu_usage                                                       0.99
max_ram_usage                                                  1436.714844
brand_raw                         Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz
count                                                                   12
l2_cache_size                                                          1.5
l3_cache_size                                                         12.0
l2_cache_line_size                                                     256
l2_cache_associativity                                                   6
benchmark                                                              KNP
ghz_actua

In [39]:
# multi thread model
print("Validation set multi thread model")
min_instance, max_instance, predictions = describe_val(model_mm, X_mm_test, y_mm_test, y_mm_scaler)
y_scaled = inv_scaling(y_mm_test, y_mm_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(mm_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(mm_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set multi thread model
Mean prediction: 14.043909072875977 | Std actual: 0.5976359248161316
Mean actual: 36.534 | Std actual: 0.8569854141115829
Mean Error: 22.49009060923258 | Std Error: 1.0447931381312872
Min instance
total_time                                                        6.79
total_cpu_usage                                                  10.72
max_ram_usage                                              2383.351562
brand_raw                         12th Gen Intel(R) Core(TM) i5-12400F
count                                                               12
l2_cache_size                                                      7.5
l3_cache_size                                                     18.0
l2_cache_line_size                                                1280
l2_cache_associativity                                               7
benchmark                                                  MATRIX_MULT
ghz_actual_friendly                                        